In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import pickle
import numpy as np
import time

# Carga los datos desde los archivos pickle
pickle_in = open("X.pickle", "rb")
X = np.array(pickle.load(pickle_in))

pickle_in = open("y.pickle", "rb")
y = np.array(pickle.load(pickle_in))

# Normaliza los datos
X = X / 255.0

dense_layers = [1]
layer_sizes = [32]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(512))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            
            model.fit(X, y, batch_size=32, epochs=7, validation_split=0.3, callbacks=[tensorboard])
            
model.save('64x3-CNN.keras')  # Cambiado a .keras


3-conv-32-nodes-1-dense-1718583705


c:\Users\Aldo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.5630 - loss: 0.6737 - val_accuracy: 0.6867 - val_loss: 0.5932
Epoch 2/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7039 - loss: 0.5736 - val_accuracy: 0.7365 - val_loss: 0.5243
Epoch 3/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7506 - loss: 0.5073 - val_accuracy: 0.7569 - val_loss: 0.4893
Epoch 4/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7776 - loss: 0.4679 - val_accuracy: 0.7798 - val_loss: 0.4660
Epoch 5/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7987 - loss: 0.4365 - val_accuracy: 0.7906 - val_loss: 0.4428
Epoch 6/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8186 - loss: 0.4001 - val_accuracy: 0.7996 - val_loss: 0.4422
Epoch 7/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8422 - loss: 0.3587 - val_accuracy: 0.7580 - val_loss: 0.5221


In [2]:
import cv2
import tensorflow as tf
import numpy as np

CATEGORIES = ["Dog", "Cat"]  # Lista de categorías, se usará para convertir el número de predicción en una etiqueta de cadena

def preparar_imagen(filepath):
    IMG_SIZE = 50  # Tamaño al que se redimensionará la imagen (50x50 en este caso)
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Cargar la imagen en escala de grises
    
    # Verificar si la imagen se cargó correctamente
    if img_array is None:
        raise ValueError("Error: No se pudo cargar la imagen. Verifica la ruta de la imagen.")
    
    # Redimensionar la imagen manteniendo la relación de aspecto original
    aspect_ratio = img_array.shape[1] / img_array.shape[0]
    if aspect_ratio > 1:  # Si la imagen es más ancha que alta
        new_width = IMG_SIZE
        new_height = int(IMG_SIZE / aspect_ratio)
    else:  # Si la imagen es más alta que ancha o cuadrada
        new_height = IMG_SIZE
        new_width = int(IMG_SIZE * aspect_ratio)
    resized_img = cv2.resize(img_array, (new_width, new_height))
    
    # Rellenar o recortar la imagen para que sea de tamaño 50x50
    padded_img = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)
    if aspect_ratio > 1:  # Si la imagen es más ancha que alta
        start_y = (IMG_SIZE - new_height) // 2
        padded_img[start_y:start_y + new_height, :] = resized_img
    else:  # Si la imagen es más alta que ancha o cuadrada
        start_x = (IMG_SIZE - new_width) // 2
        padded_img[:, start_x:start_x + new_width] = resized_img
    
    # Normalizar la imagen
    padded_img = padded_img / 255.0  # Escalar los valores de píxeles al rango [0, 1]
    padded_img = padded_img.astype('float32')  # Asegurar que los valores sean de tipo float32
    
    return padded_img.reshape(1, IMG_SIZE, IMG_SIZE, 1)  # Devolver la imagen con la forma (1, 50, 50, 1)

# Cargar el modelo pre-entrenado
ruta = (r"D:\Mis Documentos\Documentos\Aldo\Universidad\CETI\6to_Semestre\Inteligencia Artificial\3er Parcial\Proyecto\AI_Project_Deep-Learning-Tensorflow-Keras-main\64x3-CNN.keras")
modelo = tf.keras.models.load_model(ruta)

# Realizar una predicción en una imagen
ruta2 = (r"D:\Mis Documentos\Documentos\Aldo\Universidad\CETI\6to_Semestre\Inteligencia Artificial\3er Parcial\Proyecto\AI_Project_Deep-Learning-Tensorflow-Keras-main\IMG\cat.jpg")
prediction = modelo.predict(preparar_imagen(ruta2))

# Imprimir la predicción
print(prediction)  # Esto será una lista dentro de otra lista
category_index = int(prediction[0][0] >= 0.5)  # Usar el umbral 0.5 para determinar la categoría
print(CATEGORIES[category_index])  # Convertir la predicción en una etiqueta y mostrarla


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.99623036]]
Cat
